In [ ]:
!pip install git+https://github.com/openai/whisper.git

In [ ]:
!pip install torch==2.1.0 torchdata==0.7.0 torchtext==0.16.0 torchvision==0.16.0+cu118 openai-whisper==20230918

In [ ]:
pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118

In [14]:
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

In [4]:
import torch

# Print the CUDA version of PyTorch
print(torch.version.cuda)

11.8


In [12]:
!pip install torchvision==0.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 1.3 MB/s eta 0:00:00
Reason for being yanked: So that users won't accidentally install this when using python 3.11
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.16.0+cu118
    Uninstalling torchvision-0.16.0+cu118:
      Successfully uninstalled torchvision-0.16.0+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.13 requires torchvision>=0.11, but you have torchvision 0.2.0 which is incompatible.


In [15]:
import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/usr/local/lib/python3.10/dist-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


(…)a-voxceleb/resolve/main/hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

(…)eleb/resolve/main/mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

(…)-voxceleb/resolve/main/label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

In [5]:
num_speakers = 2 #@param {type:"integer"}

language = 'English' #@param ['any', 'English']

model_size = 'large-v2' #@param ['tiny', 'base', 'small', 'medium', 'large','large-v2']


model_name = model_size
if language == 'English' and model_size != 'large-v2':
  model_name += '.en'


In [6]:
import whisper
model = whisper.load_model(model_size)

100%|██████████████████████████████████████| 2.87G/2.87G [00:28<00:00, 108MiB/s]


In [7]:
result = model.transcribe('agentinsights.wav')
segments = result["segments"]

In [8]:
import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np
with contextlib.closing(wave.open('agentinsights.wav','r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

In [9]:
path = 'agentinsights.wav'

In [16]:
audio = Audio()

def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  return embedding_model(waveform[None])

In [17]:
import numpy as np

embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
    start = segment["start"]
    end = min(duration, segment["end"])
    clip = Segment(start, end)
    waveform, sample_rate = audio.crop(path, clip)

    # Ensure the waveform is mono (one channel)
    if waveform.shape[0] > 1:
        waveform = waveform.mean(axis=0)  # Convert to mono by taking the mean of channels

    # Reshape the waveform to match the expected shape of the embedding model
    waveform = waveform.reshape(1, 1, -1)

    embeddings[i] = embedding_model(waveform)

embeddings = np.nan_to_num(embeddings)


In [18]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [19]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("transcript.txt", "w")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:] + ' ')
f.close()

In [20]:
print(open('transcript.txt','r',encoding="UTF-8").read())


SPEAKER 1 0:00:00
Hello, hello, yeah, hi. 
SPEAKER 2 0:00:07
So this is actually I'm calling from you have made an query on Facebook regarding travel booking to Manali, right? 
SPEAKER 1 0:00:16
Yeah, yeah, but can I know the details like for which date are you looking for the booking? I'm looking for a week forward from now for my friends and me. How many people? 
SPEAKER 2 0:00:33
Total five including me. 
SPEAKER 1 0:00:35
OK, and can you confirm the dates? It will be one week from now. 
SPEAKER 2 0:00:39
It will be one week from now. So today 16, 16 plus 7 that is 23 around like 23, 24 right? 
SPEAKER 1 0:00:44
Yeah, around 24. 24 OK. Four days and five nights. 
SPEAKER 2 0:00:50
OK, for four days, that is five days and four nights, right? 
SPEAKER 1 0:00:52
Yeah. This is how it works. OK, so can I know your preferences? 
SPEAKER 2 0:00:57
Like what kind of rules are you looking for? Is it like you're looking for hotels 
SPEAKER 1 0:01:02
or backpacker hostels or kind of a resort?

In [22]:
import json

# Read the content from the text file
with open('transcript.txt', 'r', encoding='UTF-8') as file:
    text_content = file.read()

# Create a JSON object with the content
json_data = {
    "transcript": text_content
}

# Specify the output JSON file name
output_file = 'transcript.json'

# Save the JSON object to the JSON file
with open(output_file, 'w', encoding='UTF-8') as json_file:
    json.dump(json_data, json_file, ensure_ascii=False)

# Print a message to confirm the save
print(f'Content has been saved to {output_file}')


Content has been saved to transcript.json
